In [1]:
# Importing libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# Hyper-parameters
num_epochs = 10
batch_size = 4
learning_rate = 0.01
# Dataset has PILImage images of range [0,1]
# We transform them to Tensors of normalized range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
n_total_steps = len(train_loader)

cuda


100%|██████████| 170498071/170498071 [00:13<00:00, 12208100.95it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
# Making a convolutional layer - 2
class ConvolutionalLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, stride=1):
        super(ConvolutionalLayer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.weights = nn.Parameter(nn.init.xavier_uniform_(torch.randn(out_channels, in_channels, kernel_size, kernel_size)))
        self.biases = nn.Parameter(torch.randn(out_channels))

    def forward(self, x):
        return F.conv2d(x, self.weights, bias=self.biases, stride=self.stride, padding=self.padding)

# CNN - 1
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = ConvolutionalLayer(3, 16, 3, padding = 1) # First convolutional layer with 3 input channels (for RGB images)
        self.bn1 = nn.BatchNorm2d(16)  # Batch normalization to improve training stability
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer
        self.conv2 = ConvolutionalLayer(16, 32, 3, padding = 1) # Second convolutional layer with 16 input channels and 32 output channels.
        self.bn2 = nn.BatchNorm2d(32)

        # Fully connected layers (dense layers)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

In [16]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  total_loss = 0.0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}')

print('Finished Training')

Epoch 1, Loss: 1.5911
Epoch 2, Loss: 1.2234
Epoch 3, Loss: 1.0764
Epoch 4, Loss: 0.9898
Epoch 5, Loss: 0.9236
Epoch 6, Loss: 0.8651
Epoch 7, Loss: 0.8181
Epoch 8, Loss: 0.7763
Epoch 9, Loss: 0.7395
Epoch 10, Loss: 0.6995
Finished Training


In [19]:
# Model Testing
correct = 0
total = 0
with torch.no_grad():
  for batch in test_loader:
    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _,predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy2 = (correct / total)*100
print(accuracy2, "%")

RuntimeError: ignored